# Clean and format Ademe data

Imports

In [ ]:
import pandas as pd

### Load and concat data

In [45]:
existant = pd.read_csv('data/existant74.csv')
neuf = pd.read_csv('data/neuf74.csv')

/var/folders/vz/krrs9qks6ms_0h7qvdrthfz00000gn/T/ipykernel_651/4084556357.py:1: DtypeWarning: Columns (211,213,214,216,221,222,224,225) have mixed types. Specify dtype option on import or set low_memory=False.
  existant = pd.read_csv('data/existant74.csv')
/var/folders/vz/krrs9qks6ms_0h7qvdrthfz00000gn/T/ipykernel_651/4084556357.py:2: DtypeWarning: Columns (105,106,107,124,141,142,143,144,147,148,150,151,152,153,154,155,156,157,158,159,161,162,164,166,168,169,170,172,173,174,175,176,177,178,179,182,183,184) have mixed types. Specify dtype option on import or set low_memory=False.
  neuf = pd.read_csv('data/neuf74.csv')


Add `data_source` column to differenciate existings from new

In [46]:
existant['data_source'] = 'existant'
neuf['data_source'] = 'existant'

Concat

In [57]:
df = pd.concat([existant, neuf], ignore_index=True)

In [58]:
print('existant', len(existant.columns))
print('neuf', len(neuf.columns))
print('df:', len(df.columns))

existant 227
neuf 186
df: 231


### Check NAs

In [59]:
df.isna().sum()

configuration_installation_chauffage_n1       38215
conso_chauffage_installation_chauffage_n1     38216
type_generateur_n1_ecs_n1                     38562
numero_voie_ban                               38258
score_ban                                         0
                                              ...  
data_source                                       0
besoin_ecs_batiment                          198487
besoin_ecs_logement                          198487
deperditions_totales_logement                199048
deperditions_totales_batiment                199048
Length: 231, dtype: int64

### Add data from `communes-france-2025.csv`

Load communes and keep only interesting columns

In [60]:
communes = pd.read_csv('../web/application/datasets/communes-france-2025.csv')
communes = communes[['code_insee', 'population', 'superficie_km2', 'densite', 'altitude_moyenne', 'grille_densite_texte']]

/var/folders/vz/krrs9qks6ms_0h7qvdrthfz00000gn/T/ipykernel_651/3713826944.py:1: DtypeWarning: Columns (1,12,14,16,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  communes = pd.read_csv('../web/application/datasets/communes-france-2025.csv')


Concat data and check for non match

In [61]:
merged_df = pd.merge(df, communes, left_on='code_insee_ban', right_on='code_insee', how='left', indicator=True)
unmatched_rows = merged_df[merged_df['_merge'] == 'left_only']

### Fix non matched

Annecy makes a mess, little cities got grouped with Annecy, being now called CA Grand Annecy

In [62]:
old_annecy_insees = unmatched_rows['code_insee_ban'].unique()
old_annecy_insees

array([74011, 74268, 74187, 74093], dtype=object)

Drop Montmin 74187 (318 peoples). , It's a new city created after the fusion. Replace `old_annecy_insees` with Annecy insee 74010

In [63]:
df = df[df['code_insee_ban'] != 74187] # Drop Montmin
df['code_insee_ban'] = df['code_insee_ban'].replace(old_annecy_insees, 74010) # replace old insee to new insee

Remerge 

In [64]:
merged_df = pd.merge(df, communes, left_on='code_insee_ban', right_on='code_insee', how='left')
merged_df = merged_df.drop(columns=['code_insee'])

### Save CSV

In [65]:
merged_df.to_csv('data/merged74.csv', index=False)
merged_df.shape

(203797, 236)